In [1]:
import requests 
import os

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [3]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [37]:
from mistralai import Mistral

api_key = os.environ["MISTRAL_API_KEY"]
mistral_client = Mistral(api_key=api_key)

In [50]:
def llm(prompt):
    
    model = "mistral-small-2503"
    response = mistral_client.chat.complete(
    model= model,
    messages = [
        {
            "role": "user",
                "content": prompt,
            },
        ]
    )
    
    return response.choices[0].message.content

In [46]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [6]:
## RAG with Vector Search

In [7]:
from qdrant_client import QdrantClient, models

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
qd_client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [9]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [10]:
collection_name = "zoomcamp-faq"

In [11]:
qd_client.delete_collection(collection_name=collection_name)

False

In [12]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY, 
        distance=models.Distance.COSINE  
    )
)

True

In [30]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword" # exact matching on string metadata fields
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [31]:
points = []


for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [32]:
points[0]

PointStruct(id=0, vector=Document(text="Course - When will the course start? The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", model='jinaai/jina-embeddings-v2-small-en', options=None), payload={'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with ann

In [16]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [18]:
question = 'I just discovered the course. Can i still join it?'

In [34]:
def vector_search(question):
    course = 'data-engineering-zoomcamp'
    query_points = qd_client.query_points(
            collection_name=collection_name,
            query=models.Document(
                text=question,
                model=model_handle 
            ),
        query_filter=models.Filter( # filter by course name
                must=[
                    models.FieldCondition(
                        key="course",
                        match=models.MatchValue(value=course)
                    )
                ]
            ),
            limit=5,
            with_payload=True
        )
    
    results = []
    
    for point in query_points.points:
        results.append(point.payload)

    return results

In [35]:
vector_search('How do i run kafka?')

[{'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'For example, when running JsonConsumer.java, got:\nConsuming form kafka started\nRESULTS:::0\nRESULTS:::0\nRESULTS:::0\nOr when running JsonProducer.java, got:\nException in thread "main" java.util.concurrent.ExecutionException: org.apache.kafka.common.errors.SaslAuthenticationException: Authentication failed\nSolution:\nMake sure in the scripts in src/main/java/org/example/ that you are running (e.g. JsonConsumer.java, JsonProducer.java), the StreamsConfig.BOOTSTRAP_SERVERS_CONFIG is the correct server url (e.g. europe-west3 from example vs europe-west2)\nMake sure cluster key and secrets are updated in src/main/java/org/example/Secrets.java (KAFKA_CLUSTER_KEY and KA

In [42]:
def rag(query):
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [52]:
print(rag('how do i run kafka?'))

To run Kafka, you need to follow these steps based on the context provided:

1. **For Java Kafka:**
   - Navigate to the project directory.
   - Run the following command in the terminal:
     ```sh
     java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java
     ```
   - Ensure that the `StreamsConfig.BOOTSTRAP_SERVERS_CONFIG` in your scripts (e.g., `JsonConsumer.java`, `JsonProducer.java`) is set to the correct server URL.
   - Make sure the cluster key and secrets are updated in `src/main/java/org/example/Secrets.java` (e.g., `KAFKA_CLUSTER_KEY` and `KAFKA_CLUSTER_SECRET`).

2. **For Python Kafka:**
   - Create a virtual environment and install the required packages:
     ```sh
     python -m venv env
     source env/bin/activate  # Use `env/Scripts/activate` on Windows
     pip install -r ../requirements.txt
     ```
   - Activate the virtual environment every time you need to run the Python files:
     ```sh
     source env/bin/activate  # 